kappazunder bildstandorte vom stadt wien wfs server laden und als points.gpkg ablegen


In [ ]:
import geopandas as gpd

points = gpd.read_file("points.gpkg", layer="kappazunder_image_punkte")

In [2]:
import re

def to_snake_case(s):
    s = s.strip()               
    s = s.lower()            
    s = re.sub(r'[ -]+', '_', s)
    s = re.sub(r'[^\w-]', '', s)
    return s

points.columns = [to_snake_case(col) for col in points.columns]

timestamps der datenpunkte haben nur sekundenauflösung.  
für streng monoton steigende timestamps aufeinanderfolgende datenpunkte mit gleichen timestamps in 100ms schritten separieren.


In [3]:
import pandas as pd

points = points.sort_values(['trajectoryid', 'epoch', 'objectid']).reset_index(drop=True)

# Add milliseconds offset within each (trajectoryid, epoch) group
points['ms_offset'] = points.groupby(['trajectoryid', 'epoch']).cumcount() * 100

# Apply offset as timedelta (1 ms per duplicate)
points['epoch'] = points['epoch'] + pd.to_timedelta(points['ms_offset'], unit='ms')

# Drop helper column
points = points.drop(columns='ms_offset')

bildstandorte sind in epsg:31256, später brauchen wir zusätzlich wsg84


In [4]:
points_wsg84 = points.to_crs(epsg=4326)
points["lat"] = points_wsg84.geometry.y
points["lon"] = points_wsg84.geometry.x

trajectoryid als string interpretieren


In [5]:
points['trajectoryid'] = points['trajectoryid'].astype(str).str.replace('.0', '')
print("New trajectoryid dtype:", points['trajectoryid'].dtype)
print("\nSample of trajectoryid values:")
print(points['trajectoryid'].head())

New trajectoryid dtype: object

Sample of trajectoryid values:
0    15419
1    15419
2    15419
3    15419
4    15419
Name: trajectoryid, dtype: object


der geodatenviewer erlaubt maximal 80gb große downloads.

bei guten 20mb pro datenpunkt macht das etwa 3500 datenpunkten pro download.

die günstige vm bei hetzner hat nur 40gb ssd. außerdem muss das heruntergeladene archiv entpackt werden.
ein archiv muss also etwas unter 20gb haben. damit sind wir bei unter 1000 datenpunkten pro download.

die eine trajektorie umfassenden polygone werden zu einem gewissen grad auch trajektorie-fremde datenpunkte umschließen was den download vergrößert. deswegen pro download paket maximal 750 datenpunkte.


In [6]:
print(f"{points['trajectoryid'].nunique()} Trajektorien")
print("Datenpunkte pro Trajektorie:")
print(points["trajectoryid"].value_counts())

175 Trajektorien
Datenpunkte pro Trajektorie:
trajectoryid
15728    21114
16442    20582
15433    20311
15419    20106
16653    19793
         ...  
17720       56
16101       40
16471       28
15432        3
17096        2
Name: count, Length: 175, dtype: int64


die trajektorien müssen also aufgeteilt werden.

die datenpunkte jeder trajektorie chronologisch sortieren über die objectid und in untertrajektorien mit maximal 750 datenpunkten aufteilen.


In [7]:
from math import ceil
from tqdm import tqdm

original_trajectories_count = points['trajectoryid'].nunique()

records = []
for traj_id, group in tqdm(points.groupby('trajectoryid')):
    group_sorted = group.sort_values('objectid')
    n = len(group_sorted)
    if n == 0:
        continue
    no_of_splits = int(ceil(n / 750))
    base = n // no_of_splits
    remainder = n % no_of_splits
    sizes = [base + 1] * remainder + [base] * (no_of_splits - remainder)
    i = 0
    for split_i, size in enumerate(sizes):
        end = i + size
        part = group_sorted.iloc[i:end].copy()
        part['trajectoryid'] = f"{traj_id}_{split_i}" if len(sizes) > 1 else traj_id
        records.append(part)
        i = end

points_split = gpd.GeoDataFrame(pd.concat(records, ignore_index=True), crs=points.crs)

print(f"Original trajectories: {original_trajectories_count}")
print(f"Split trajectories: {points_split['trajectoryid'].nunique()}")
print("Counts per split:")
print(points_split['trajectoryid'].value_counts())

100%|██████████| 175/175 [00:00<00:00, 183.51it/s]


Original trajectories: 175
Split trajectories: 1917
Counts per split:
trajectoryid
17562_1    750
21441_3    750
16876_2    750
16876_1    750
16876_0    750
          ... 
17720       56
16101       40
16471       28
15432        3
17096        2
Name: count, Length: 1917, dtype: int64


der geodatenviewer benötigt einen die gewünschten datenpunkte umfassenden polygon.

dazu die datenpunkte jeder trajektorie chronologisch zu einem linestring verbinden und diesen auf 1 meter dicke buffern.


In [8]:
from shapely.geometry import LineString

records = []
for traj_id, group in tqdm(points_split.groupby('trajectoryid')):
    group_sorted = group.sort_values('objectid')
    geoms = list(group_sorted.geometry)
    if len(geoms) == 0:
        continue
    if len(geoms) == 1:
        # single point trajectory -> keep point (buffer will still work)
        line_geom = geoms[0]
    else:
        coords = [(pt.x, pt.y) for pt in geoms]
        line_geom = LineString(coords).simplify(0.1)
    records.append({'trajectoryid': traj_id, 'geometry': line_geom})

# GeoDataFrame of lines (or single-point geometries for 1-point trajectories)
trajectory_lines = gpd.GeoDataFrame(records, crs=points_split.crs)

# Buffer by 0.5 meter (CRS is EPSG:31256 so units are meters)
trajectories = trajectory_lines.copy()
trajectories['geometry'] = trajectories.geometry.buffer(0.5, cap_style=3, join_style=2)

print("Trajectories (lines):", len(trajectory_lines))
print("Buffered geometries:", len(trajectories))

# Optionally inspect first rows
trajectory_lines.head(), trajectories.head()

100%|██████████| 1917/1917 [00:06<00:00, 292.70it/s]


Trajectories (lines): 1917
Buffered geometries: 1917


(  trajectoryid                                           geometry
 0      15419_0  LINESTRING (-601.117 337164.375, -639.231 3371...
 1      15419_1  LINESTRING (-2268.27 337022.539, -2274.858 337...
 2     15419_10  LINESTRING (24.193 337690.697, 22.957 337708.5...
 3     15419_11  LINESTRING (293.683 338439.65, 303.067 338424....
 4     15419_12  LINESTRING (-150.774 337586.244, -149.835 3375...,
   trajectoryid                                           geometry
 0      15419_0  POLYGON ((-639.122 337155.512, -674.282 337147...
 1      15419_1  POLYGON ((-2490.5 337001.286, -2492.318 337002...
 2     15419_10  POLYGON ((18.156 337814.436, 19.032 337800.925...
 3     15419_11  POLYGON ((-368.22 338254.55, -367.019 338255.3...
 4     15419_12  POLYGON ((-149.341 337577.431, -148.629 337574...)

aufbereitete datenpunkte und trajektorie-polygone abspeichern.


In [ ]:
points.to_file(
    "points.gpkg", driver="GPKG", layer="kappazunder_image_punkte", index=False
)
trajectories.to_file("trajectories.gpkg", driver="GPKG", index=False)